In [38]:
import numpy as np
from selenium import webdriver
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as p
import string
import time

In [39]:
path = "E:/Virtual environment/Recommender_Systems/ml-latest-small/"
movies = pd.read_csv(path + 'movies.csv')

In [40]:
links = pd.read_csv(path + 'links.csv')
URLs = links['imdbId']
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [41]:
search_string = "https://www.imdb.com/title/tt0"
IMDBS = URLs.values.tolist()
URL_list = []

for i in IMDBS:
    i = str(i)
    final_URL = search_string + i + "/"
    URL_list.append(final_URL)

print(len(URL_list))

9742


In [42]:
URLS = URL_list[0:500]

In [43]:
titles = []
release_years = []
age_classifications = []
runtimes = []
rates = []
genres = []
movie_stories = []
directors = []
writerss = []
starss = []

# create a web driver
driver = webdriver.Chrome(ChromeDriverManager().install())

# iterate over each URL and scrape the data
for url in URLS:
    try:
        # load the URL in the Chrome webdriver
        driver.get(url)
        content = driver.page_source
        soup = p(content)
        driver.implicitly_wait(10)
        
        title = soup.find('h1')
        titles.append(title.text)
        
        x = soup.find('ul', class_='ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt')
        y = x.find_all('a')
        release_years.append(str(y[0].text))
        try:
            age_classifications.append(str(y[1].text))
        except:
            age_classifications.append(None)
        
        #runtime = soup.find_all('li', '<!-- -->', role = 'presentation', class_='ipc-inline-list__item')
        #runtimes.append(str(runtime[5].text))
        
        runtime_elem = soup.find('li', {'data-testid': 'title-techspec_runtime'})
        if runtime_elem is not None:
            runtime = runtime_elem.find('div', {'class': 'ipc-metadata-list-item__content-container'})
            if runtime is not None:
                runtimes.append(runtime.text.strip())
            else:
                runtimes.append('')
        else:
            runtimes.append('')
        
        rate = soup.find('span', class_ = 'sc-bde20123-1 iZlgcd')
        rates.append(rate.text)
        
        genre_tags = soup.find_all('a', class_='ipc-chip ipc-chip--on-baseAlt')
        a = ''
        for i in genre_tags:
            a+=i.text+', '
        genres.append(a[:len(a)-2])
        
        movie_story = soup.find('span', class_='sc-6a7933c5-0 cUeLJx')
        movie_stories.append(movie_story.text)

        credit_items = soup.find_all('li', {'class': 'ipc-metadata-list__item', 'data-testid': 'title-pc-principal-credit'})
        directors.append(credit_items[0].find('a').text)
        writers = []
        stars = []
        
        for item in credit_items[1:3]:
            people = item.find('ul')
            x = people.find_all('a')
            if 'Writer' in item.text:
                for person in x:
                    writers.append(person.text)
            elif 'Stars' in item.text:
                for person in x:
                    stars.append(person.text)
        writerss.append(', '.join(writers))
        starss.append(', '.join(stars))
    except Exception as e:
        print(f"Error occurred while scraping data from {url}: {e}")
    
# store the data in a pandas DataFrame
df = pd.DataFrame({
    'Title': titles,
    'Production Year': release_years,
    'Age_classifications': age_classifications,
    'IMDB score': rates,
    'Runtime': runtimes,
    'Genres': genres,
    'Synopsis': movie_stories,
    'Director' : directors,
    'Writers' : writerss,
    'Stars': starss
})

# close the web driver
driver.quit()

C:\Users\ZRS\AppData\Local\Temp\ipykernel_21924\3444003071.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Error occurred while scraping data from https://www.imdb.com/title/tt075314/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt061395/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt058450/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt076759/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt059170/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt074102/: 'NoneType' object has no attribute 'text'
Error occurred while scraping data from https://www.imdb.com/title/tt083658/: 'NoneType' object has no attribute 'text'


In [44]:
df

,Title,Production Year,Age_classifications,IMDB score,Runtime,Genres,Synopsis,Director,Writers,Stars
0,Toy Story,1995,G,8.3,1 hour 21 minutes,"Animation, Adventure, Comedy",A cowboy doll is profoundly threatened and jea...,John Lasseter,"John Lasseter, Pete Docter, Andrew Stanton","Tom Hanks, Tim Allen, Don Rickles"
1,Jumanji,1995,PG,7.0,1 hour 44 minutes,"Adventure, Comedy, Family",When two kids find and play a magical board ga...,Joe Johnston,"Jonathan Hensleigh, Greg Taylor, Jim Strain","Robin Williams, Kirsten Dunst, Bonnie Hunt"
2,Grumpier Old Men,1995,PG-13,6.6,1 hour 41 minutes,"Comedy, Romance",John and Max resolve to save their beloved bai...,Howard Deutch,Mark Steven Johnson,"Walter Matthau, Jack Lemmon, Ann-Margret"
3,Waiting to Exhale,1995,R,5.9,2 hours 4 minutes,"Comedy, Drama, Romance","Based on Terry McMillan's novel, this film fol...",Forest Whitaker,"Terry McMillan, Ron Bass","Whitney Houston, Angela Bassett, Loretta Devine"
4,Father of the Bride Part II,1995,PG,6.0,1 hour 46 minutes,"Comedy, Family, Romance",George Banks must deal not only with his daugh...,Charles Shyer,"Albert Hackett, Frances Goodrich, Nancy Meyers","Steve Martin, Diane Keaton, Martin Short"
...,...,...,...,...,...,...,...,...,...,...
488,"Ciao, Professore!",1992,R,6.9,1 hour 40 minutes,Comedy,A teacher must deal with the underworld of Nap...,Lina Wertmüller,"Alessandro Bencivenni, Leonardo Benvenuti, Mar...","Paolo Villaggio, Isa Danieli, Gigio Morra"
489,Spanking the Monkey,1994,Unrated,6.3,1 hour 40 minutes,"Comedy, Drama",A med student is forced by his traveling sales...,David O. Russell,David O. Russell,"Jeremy Davies, Alberta Watson, Benjamin Hendri..."
490,The Little Rascals,1994,PG,6.3,1 hour 22 minutes,"Comedy, Family, Romance","Alfalfa is wooing Darla and his ""He-Man-Woman-...",Penelope Spheeris,"Penelope Spheeris, Robert Wolterstorff, Mike S...","Travis Tedford, Bug Hall, Brittany Ashton Holmes"
491,Andre,1994,PG,5.7,1 hour 34 minutes,"Adventure, Drama, Family",The true story of how a seal named Andre befri...,George Miller,"Harry Goodridge, Lew Dietz, Dana Baratta","Tina Majorino, Keith Carradine, Chelsea Field"


In [47]:
#df.to_csv('First500.csv', index=False)
z = pd.read_csv('First500.csv')
z

,Title,Production Year,Age_classifications,IMDB score,Runtime,Genres,Synopsis,Director,Writers,Stars
0,Toy Story,1995,G,8.3,1 hour 21 minutes,"Animation, Adventure, Comedy",A cowboy doll is profoundly threatened and jea...,John Lasseter,"John Lasseter, Pete Docter, Andrew Stanton","Tom Hanks, Tim Allen, Don Rickles"
1,Jumanji,1995,PG,7.0,1 hour 44 minutes,"Adventure, Comedy, Family",When two kids find and play a magical board ga...,Joe Johnston,"Jonathan Hensleigh, Greg Taylor, Jim Strain","Robin Williams, Kirsten Dunst, Bonnie Hunt"
2,Grumpier Old Men,1995,PG-13,6.6,1 hour 41 minutes,"Comedy, Romance",John and Max resolve to save their beloved bai...,Howard Deutch,Mark Steven Johnson,"Walter Matthau, Jack Lemmon, Ann-Margret"
3,Waiting to Exhale,1995,R,5.9,2 hours 4 minutes,"Comedy, Drama, Romance","Based on Terry McMillan's novel, this film fol...",Forest Whitaker,"Terry McMillan, Ron Bass","Whitney Houston, Angela Bassett, Loretta Devine"
4,Father of the Bride Part II,1995,PG,6.0,1 hour 46 minutes,"Comedy, Family, Romance",George Banks must deal not only with his daugh...,Charles Shyer,"Albert Hackett, Frances Goodrich, Nancy Meyers","Steve Martin, Diane Keaton, Martin Short"
...,...,...,...,...,...,...,...,...,...,...
488,"Ciao, Professore!",1992,R,6.9,1 hour 40 minutes,Comedy,A teacher must deal with the underworld of Nap...,Lina Wertmüller,"Alessandro Bencivenni, Leonardo Benvenuti, Mar...","Paolo Villaggio, Isa Danieli, Gigio Morra"
489,Spanking the Monkey,1994,Unrated,6.3,1 hour 40 minutes,"Comedy, Drama",A med student is forced by his traveling sales...,David O. Russell,David O. Russell,"Jeremy Davies, Alberta Watson, Benjamin Hendri..."
490,The Little Rascals,1994,PG,6.3,1 hour 22 minutes,"Comedy, Family, Romance","Alfalfa is wooing Darla and his ""He-Man-Woman-...",Penelope Spheeris,"Penelope Spheeris, Robert Wolterstorff, Mike S...","Travis Tedford, Bug Hall, Brittany Ashton Holmes"
491,Andre,1994,PG,5.7,1 hour 34 minutes,"Adventure, Drama, Family",The true story of how a seal named Andre befri...,George Miller,"Harry Goodridge, Lew Dietz, Dana Baratta","Tina Majorino, Keith Carradine, Chelsea Field"
